In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import numba

In [3]:
#Problem 1
# This is an example to give you an idea
@tf.function(autograph=False)
def log_abs(x):
    """EXAMPLE IMPLEMENTATION: Return the log of the absolute of x element-wise"""
    return tf.math.log(tf.math.abs(x))


In [4]:
log_abs(5.0)

<tf.Tensor: shape=(), dtype=float32, numpy=1.609438>

In [5]:
#Problem 2
@tf.function(autograph=False)
def sum_cos_sin(x, coeff_cos, coeff_sin):
    """Return the sum of the cos and sin of x element-wise, cos and sin scaled by coeff_cos and coeff_sin respectively."""
    return coeff_sin * tf.math.sin(x) + coeff_cos * tf.math.cos(x)

In [6]:
sum_cos_sin(5.0,1.0,2.0)

<tf.Tensor: shape=(), dtype=float32, numpy=-1.6341864>

In [9]:
#Problem 3
@tf.function(autograph=False)
def approx_cos_p1(x):
    """Return the approximation of cos(x) + 1 using a taylor/series expansion up to order 7
    Explanation: The Taylor series expansion of cos(x) + 1 is :math:`1 + \sum_{i=0}^{i~is~even} (-1)^{i/2} x^i / i!`
    I calculate all taylor terms separately and store them along a new axis and then sum them together.
    Since all terms are stored in memory at one point, it takes up 4x the memory of x, which could be a problem.
    """
    x = tf.cast(x, dtype=tf.float64)
    x = tf.expand_dims(x, axis=-1)
    order = tf.constant(7., dtype=tf.float64)
    factorial_without_zeroth = tf.math.cumprod(tf.range(1., order+1, dtype=tf.float64))
    factorial = tf.concat((tf.constant([1.], dtype=tf.float64), factorial_without_zeroth), axis=0)
    i = tf.range(0., order+1., 2, dtype=tf.float64)

    taylor_terms = tf.constant(-1., dtype=tf.float64) ** (i/2.) / tf.gather(factorial, tf.cast(i, tf.int32)) * x**i
    return tf.math.reduce_sum(taylor_terms, axis=-1) + tf.constant(1., dtype=tf.float64)

In [10]:
approx_cos_p1(5.0)

<tf.Tensor: shape=(), dtype=float64, numpy=-6.159722222222221>

In [11]:
#Problem 4
@tf.function(autograph=False)
def integral_exp(lower, upper):
    """Integrate the exponential function from lower to upper.
    Explanation: Since the primitive function of e^x is e^x, we can simply calculate e^upper - e^lower
    """
    return tf.math.exp(upper) - tf.math.exp(lower)

In [12]:
integral_exp(5.,7.)

<tf.Tensor: shape=(), dtype=float32, numpy=948.22003>

In [13]:
#Problem 5
@tf.function(autograph=False)
def normed_exp(x, lower, upper):
    """Calculate the normalized exp function `exp(x) / integral exp from lower to upper`"""
    return tf.math.exp(x) / integral_exp(lower, upper)

In [14]:
normed_exp(2.,1.,5.)

<tf.Tensor: shape=(), dtype=float32, numpy=0.05071596>

In [15]:
#Problem 6
@tf.function(autograph=False)
def cos_exp(x):
    """Return the elementwise value of the function 'sum_cos_sin(x) for x < - 1; exp(x) for -1 < x < 3; cos(x) for x > 3.
    Explanation: I use tf.where to only insert sum_cos_sin(x) where x < -1 etc.
    One downside to the current implementation is that I calculate exp(x), cos(x), sum_cos_sin(x) for all values of x,
    resulting in 3x as many calculations as necessary.
    I tried to solve this problem by masking parts of x with nan values but that method was consistently slower.
    """
    return tf.where(x < -1, sum_cos_sin(x, 1., 1.), tf.where(x < 3, tf.math.exp(x), tf.math.cos(x)))

In [20]:
cos_exp(-2.)

<tf.Tensor: shape=(), dtype=float32, numpy=-1.3254442>

In [18]:
cos_exp(0.)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [21]:
cos_exp(4.)

<tf.Tensor: shape=(), dtype=float32, numpy=-0.6536436>

In [22]:
#Problem 7
# use @tf.custom_gradient to add a custom gradient using again a series approximation
@tf.function(autograph=False)
@tf.custom_gradient
def approx_cos_p1_custom_grad(x):
    """Return the approximation of cos(x) + 1 using a taylor/series expansion up to order 7 **with a custom gradient**
    Explanation: the lines outside cos_p1_gradient (besides the first line) are taken from approx_cos_p1.
    The explanation for the gradient is given in its own docstring.
    """
    y = approx_cos_p1(x)

    x = tf.cast(x, dtype=tf.float64)
    x = tf.expand_dims(x, axis=-1)
    order = tf.constant(7., dtype=tf.float64)
    factorial_without_zeroth = tf.math.cumprod(tf.range(1., order+1, dtype=tf.float64))
    factorial = tf.concat((tf.constant([1.], dtype=tf.float64), factorial_without_zeroth), axis=0)

    @tf.function(autograph=False)
    def cos_p1_gradient(dy):
        """Return the gradient of the taylor series of cos(x)+1, i.e. the taylor series of -sin(x).
        Explanation: The Taylor series expansion of cos(x) + 1 is :math:`\sum_{i=1}^{i~is~odd} (-1)^{(i+1)/2} x^i / i!`
        Here, I use the 7th order Taylor series of -sin(x). If we want it to be the gradient of the taylor series of cos(x)+1,
        we can simply replace order+1 with order in the first line below.
        """
        i = tf.range(1., order+1, 2, dtype=tf.float64)
        taylor_terms = tf.constant(-1., dtype=tf.float64) ** ((i+1)/2.) / tf.gather(factorial, tf.cast(i, tf.int32)) * x**i
        return dy * tf.math.reduce_sum(taylor_terms, axis=-1)

    return y, cos_p1_gradient

In [28]:
#Problem 8
from abc import ABC, abstractmethod
class Func(ABC):
    @abstractmethod
    def value(self, x):
        pass

    @abstractmethod
    def integral(self, lower, upper):
        pass

class CosFunc(Func):
    def __init__(self, omega):
        self._omega = omega

    def value(self, x):
        """Calculate the value of cos(omega*x)"""
        return tf.math.cos(self._omega * x)

    def integral(self, lower, upper):
        """Calculate the integral of cos(omega*x) from lower to upper.
        Here, I use the fact that the primitive function of cos(omega*x) is sin(omega*x)/omega"""
        return (tf.math.sin(self._omega * upper) - tf.math.sin(self._omega * lower)) / self._omega


# create a class CosPDF, which inherits from PDF and takes two parameters to instantiate, `lower` and `upper`.
# Then implement the `normed_value` which is the normalized value: value / integral
# Code provided
class PDF(Func):
    @abstractmethod
    def normed_value(self, x):
        pass

class CosPDF(PDF, CosFunc):
    def __init__(self, lower, upper):
        # Call CosFunc.__init__ with omega=1
        super().__init__(tf.constant(1., dtype=tf.float64))
        self._lower = lower
        self._upper = upper

    def normed_value(self, x):
        """Return the normalized value of cos(x)"""
        return self.value(x) / self.integral(self._lower, self._upper)


if __name__ == '__main__':
    # test the functions here
    # it should work for both `x`
    use_single_val = True
    if use_single_val:
        x = tf.constant(1., dtype=tf.float64)
        lower = tf.constant(0.3, dtype=tf.float64)
        upper = tf.constant(5., dtype=tf.float64)
        coeff_sin = tf.constant(3.1, dtype=tf.float64)
        coeff_cos = tf.constant(2.1, dtype=tf.float64)
    else:
        shape = (100, 30)
        x = tf.random.uniform(shape=shape, minval=-4.5, maxval=2.3)
        lower = tf.random.uniform(shape=shape, minval=-5.5, maxval=-5.3)
        upper = tf.random.uniform(shape=shape, minval=4.1, maxval=5.2)
        coeff_sin = tf.random.uniform(shape=shape, minval=3.1, maxval=5.2)
        coeff_cos = tf.random.uniform(shape=shape, minval=1.2, maxval=2.23)

    # Test that all functions run without raising any errors
    functions = [{'function': sum_cos_sin, 'args': [x, coeff_cos, coeff_sin]},
                 {'function': approx_cos_p1, 'args': [x]},
                 {'function': integral_exp, 'args': [lower, upper]},
                 {'function': normed_exp, 'args': [x, lower, upper]},
                 {'function': cos_exp, 'args': [x]}]

    for test_function_config in functions:
        test_function_config['function'](*test_function_config['args'])

    # Test that the custom gradient can be calculated without raising any errors
    with tf.GradientTape() as tape:
        tape.watch(x)
        y = approx_cos_p1_custom_grad(x)
    grad = tape.gradient(y, x)

    # test the class here
    # Arbitrarily chosen constants
    omega = tf.constant(2., dtype=tf.float64)
    lower = tf.constant(-1., dtype=tf.float64)
    upper = tf.constant(8.7, dtype=tf.float64)
    x = tf.constant(3., dtype=tf.float64)

    cos_func = CosFunc(omega=omega)
    # Value should be ~ cos(2. * 3.)
    assert tf.abs(cos_func.value(x) - tf.math.cos(omega * x)) < 1e-10
    # Check that the value of the integral matches the expected answer
    #  (Value taken from https://www.wolframalpha.com/input/?i=integral+cos%282x%29+from+-1+to+8.7)
    assert tf.abs(cos_func.integral(lower, upper) - (-0.041681)) < 1e-5

    cos_pdf = CosPDF(lower, upper)
    # Check that the normed value matches the expected answer
    #  (Value taken from https://www.wolframalpha.com/input/?i=cos%283%29+%2F+%28integral+cos%28x%29+from+-1+to+8.7%29)
    assert tf.abs(cos_pdf.normed_value(x) - (-0.658047)) < 1e-5
